<a href="https://colab.research.google.com/github/ptmhoang97/train_dataset/blob/main/train_model_transferlearning_tf1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Sep 23 07:42:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
batch_size_pipeline = 16
num_steps_training = 10000
learning_rate_pipeline = 0.1
warmup_learning_rate_pipeline = learning_rate_pipeline/10
models_drive_path = "/content/drive/MyDrive/workspaces_tensorflow/train_model_tf1"
train_name = 'next_trained_model'
model_name = 'ssd_mobilenet_v2_coco_2018_03_29'
train_dataset = "/content/drive/MyDrive/train.zip"
test_dataset = "/content/drive/MyDrive/test.zip"

In [5]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


# 0. Setup Paths

In [6]:
import os

In [7]:
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [8]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc'),
    'MODEL_PATH': os.path.join(models_drive_path),
    'CHECKPOINT_PATH': os.path.join(models_drive_path, train_name), 
    'OUTPUT_PATH': os.path.join(models_drive_path, train_name, 'export'), 
    'TFJS_PATH':os.path.join(models_drive_path, train_name, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join(models_drive_path, train_name, 'tfliteexport'), 
    'PRETRAINED_MODEL_PATH':os.path.join(models_drive_path, model_name), 
 }

In [38]:
files = {
    'PIPELINE_CONFIG':os.path.join(models_drive_path, train_name, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME),
    'PIPELINE_CONFIG_BASE':os.path.join(models_drive_path, model_name, 'pipeline.config'),
    'CHECKPOINT_BASE':os.path.join(models_drive_path, model_name, 'model.ckpt'),
}

In [10]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}
        print(path)

Tensorflow/workspace
Tensorflow/scripts
Tensorflow/models
Tensorflow/workspace/annotations
Tensorflow/workspace/images
Tensorflow/protoc
/content/drive/MyDrive/workspaces_tensorflow/train_model_tf1/next_trained_model
/content/drive/MyDrive/workspaces_tensorflow/train_model_tf1/next_trained_model/export
/content/drive/MyDrive/workspaces_tensorflow/train_model_tf1/next_trained_model/tfjsexport
/content/drive/MyDrive/workspaces_tensorflow/train_model_tf1/next_trained_model/tfliteexport


# 1. Install TF Object Dectection API

In [11]:
# https://www.tensorflow.org/install/source_windows

In [12]:
if os.name=='nt':
    !pip install wget
    import wget

In [13]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 63156, done.
remote: Counting objects: 100% (768/768), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 63156 (delta 595), reused 749 (delta 581), pack-reused 62388
Receiving objects: 100% (63156/63156), 574.78 MiB | 25.21 MiB/s, done.
Resolving deltas: 100% (44050/44050), done.


In [14]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf1/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf1\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Processing /content/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 352 kB 8.3 MB/s 
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1665484 sha256=97463b115c520a164c7f4bf46689909bed581343a07412e350f3346ededaa2c5
  Stored in directory: /tmp/pip-ephem-wheel-cache-qy_ez4ex/wheels/a9/26/bf/1cb2313ed4855917889b97658bf

In [15]:
print(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf1_test.py'))

Tensorflow/models/research/object_detection/builders/model_builder_tf1_test.py


In [16]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf1_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_model_from_config_w

In [17]:
import object_detection

# 2. Download dataset

In [18]:
!unzip {train_dataset} -d {paths['IMAGE_PATH']}

Archive:  /content/drive/MyDrive/train.zip
   creating: Tensorflow/workspace/images/train/
  inflating: Tensorflow/workspace/images/train/005980b3a5f93085_jpg.rf.2QZCXGReqcBORWQJKQfO.jpg  
  inflating: Tensorflow/workspace/images/train/005980b3a5f93085_jpg.rf.2QZCXGReqcBORWQJKQfO.xml  
  inflating: Tensorflow/workspace/images/train/005980b3a5f93085_jpg.rf.68f1ed44dbca53b707a1d3d82d4b7d9a.jpg  
  inflating: Tensorflow/workspace/images/train/005980b3a5f93085_jpg.rf.68f1ed44dbca53b707a1d3d82d4b7d9a.xml  
  inflating: Tensorflow/workspace/images/train/00aaf0a0a9ee7e71_jpg.rf.808b1e59067887493dffad63561c2a9d.jpg  
  inflating: Tensorflow/workspace/images/train/00aaf0a0a9ee7e71_jpg.rf.808b1e59067887493dffad63561c2a9d.xml  
  inflating: Tensorflow/workspace/images/train/00aaf0a0a9ee7e71_jpg.rf.KBCzX2HtZviBuQY9JDTQ.jpg  
  inflating: Tensorflow/workspace/images/train/00aaf0a0a9ee7e71_jpg.rf.KBCzX2HtZviBuQY9JDTQ.xml  
  inflating: Tensorflow/workspace/images/train/00e0422e47cdbeb7_jpg.rf.a8b6f2

In [19]:
!unzip {test_dataset} -d {paths['IMAGE_PATH']}

Archive:  /content/drive/MyDrive/test.zip
   creating: Tensorflow/workspace/images/test/
  inflating: Tensorflow/workspace/images/test/00dea1edf14f09ab_jpg.rf.3f17c8790a68659d03b1939a59ccda80.jpg  
  inflating: Tensorflow/workspace/images/test/00dea1edf14f09ab_jpg.rf.3f17c8790a68659d03b1939a59ccda80.xml  
  inflating: Tensorflow/workspace/images/test/00dea1edf14f09ab_jpg.rf.KJ730oDTFPdXdJxvSLnX.jpg  
  inflating: Tensorflow/workspace/images/test/00dea1edf14f09ab_jpg.rf.KJ730oDTFPdXdJxvSLnX.xml  
  inflating: Tensorflow/workspace/images/test/00e481ea1a520175_jpg.rf.6e6a8b3b45c9a11d106958f88ff714ea.jpg  
  inflating: Tensorflow/workspace/images/test/00e481ea1a520175_jpg.rf.6e6a8b3b45c9a11d106958f88ff714ea.xml  
  inflating: Tensorflow/workspace/images/test/00e481ea1a520175_jpg.rf.MV6sZ8QCFwFeMYaI2tHm.jpg  
  inflating: Tensorflow/workspace/images/test/00e481ea1a520175_jpg.rf.MV6sZ8QCFwFeMYaI2tHm.xml  
  inflating: Tensorflow/workspace/images/test/08c8b73e0c2e296e_jpg.rf.7IkYAamjZhnwsoXSr

# 3. Create Label Map

In [20]:
labels = [
          {'name':'Ambulance', 'id':1},
          {'name':'Bus', 'id':2},
          {'name':'Car', 'id':3},
          {'name':'Motorcycle', 'id':4},
          {'name':'Truck', 'id':5},
          ]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 4. Create TF records

In [21]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

Cloning into 'Tensorflow/scripts'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [22]:
train_dataset_folder = train_dataset[train_dataset.rfind("/")+1:train_dataset.rfind(".")]
test_dataset_folder = test_dataset[test_dataset.rfind("/")+1:test_dataset.rfind(".")]
print(train_dataset_folder)
print(test_dataset_folder)

train
test


In [23]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], train_dataset_folder)} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], test_dataset_folder)} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 5. Copy Model Config to Training Folder

In [41]:
if os.name =='posix':
    !cp {os.path.join(files['PIPELINE_CONFIG_BASE'])} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(files['PIPELINE_CONFIG_BASE'])} {os.path.join(paths['CHECKPOINT_PATH'])}

# 6. Update Config For Transfer Learning

In [42]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [43]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [44]:
config

{'eval_config': num_examples: 8000
 max_evals: 10
 use_moving_averages: false,
 'eval_input_config': label_map_path: "PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt"
 shuffle: false
 num_readers: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BE_CONFIGURED/mscoco_val.record"
 },
 'eval_input_configs': [label_map_path: "PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt"
 shuffle: false
 num_readers: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BE_CONFIGURED/mscoco_val.record"
 }
 ],
 'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 300
       width: 300
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
    

In [45]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [46]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = batch_size_pipeline
pipeline_config.train_config.num_steps = num_steps_training
pipeline_config.train_config.optimizer.rms_prop_optimizer.learning_rate.exponential_decay_learning_rate.initial_learning_rate = learning_rate_pipeline
pipeline_config.train_config.fine_tune_checkpoint = files['CHECKPOINT_BASE']
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [48]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 7. Train the model

In [49]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main.py')

In [51]:
command = "python {} --model_dir={} --pipeline_config_path={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [52]:
print(command)

python Tensorflow/models/research/object_detection/model_main.py --model_dir=/content/drive/MyDrive/workspaces_tensorflow/train_model_tf1/next_trained_model --pipeline_config_path=/content/drive/MyDrive/workspaces_tensorflow/train_model_tf1/next_trained_model/pipeline.config


In [62]:
!{command}

Instructions for updating:
Please use `layer.__call__` method instead.
W0923 09:14:02.501422 140283417143168 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0923 09:14:04.546977 140283417143168 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0923 09:14:04.585763 140283417143168 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0923 09:14:04.624355 140283417143168 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

# 8. Evaluate model
Evaluate model in training process also.

#9. Freeze graph

In [57]:
import re
import numpy as np

lst = os.listdir(paths['CHECKPOINT_PATH'])
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
last_model_path = "{}/{}".format(paths['CHECKPOINT_PATH'],last_model)
print(last_model)
print(last_model_path)

model.ckpt-10000
/content/drive/MyDrive/workspaces_tensorflow/train_model_tf1/next_trained_model/model.ckpt-10000


In [58]:
FROZEN_GRAPH_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'export_inference_graph.py')

In [59]:
command = "python {} --input_type=image_tensor  --pipeline_config_path={} --trained_checkpoint_prefix={} --output_directory={}".format(FROZEN_GRAPH_SCRIPT, files['PIPELINE_CONFIG'], last_model_path, paths['OUTPUT_PATH'])

In [60]:
print(command)

python Tensorflow/models/research/object_detection/export_inference_graph.py --input_type=image_tensor  --pipeline_config_path=/content/drive/MyDrive/workspaces_tensorflow/train_model_tf1/next_trained_model/pipeline.config --trained_checkpoint_prefix=/content/drive/MyDrive/workspaces_tensorflow/train_model_tf1/next_trained_model/model.ckpt-10000 --output_directory=/content/drive/MyDrive/workspaces_tensorflow/train_model_tf1/next_trained_model/export


In [61]:
!{command}

Instructions for updating:
Please use `layer.__call__` method instead.
W0923 09:02:26.991612 139995490592640 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0923 09:02:29.218487 139995490592640 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0923 09:02:29.254509 139995490592640 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0923 09:02:29.289250 139995490592640 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

# 10. Zip model and copy to GG Drive

In [ ]:
import datetime
import pytz

def shorten_time(date):
    dt_year = date[0:4]
    dt_month = date[5:7]
    dt_day = date[8:10]
    dt_hr = date[11:13]
    dt_min = date[14:16]
    dt_sec = date[17:19]
    dt_full = "{}{}{}_{}{}{}".format(dt_year,dt_month,dt_day,dt_hr,dt_min,dt_sec)
    return dt_full

start_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
start_date_short = shorten_time(start_date)
print(start_date)
print(start_date_short)

In [ ]:
%cd {paths['MODEL_PATH']}

In [ ]:
export_model_path = "next_trained_model"
export_model_zip = "{}_{}.zip".format(export_model_path,start_date_short)
print(export_model_path)
print(export_model_zip)

In [ ]:
!zip -r {export_model_zip} {export_model_path}

In [ ]:
%cd "/content"

# 11. Notification via Email

In [ ]:
import datetime
import pytz

finish_date = str(datetime.datetime.now(pytz.timezone('Asia/Bangkok')))[:19]
finish_date_short = shorten_time(finish_date)

print(finish_date)
print(finish_date_short)

In [ ]:
sd_year = int(start_date_short[0:4])
sd_month = int(start_date_short[4:6])
sd_day = int(start_date_short[6:8])
sd_hour = int(start_date_short[9:11])
sd_min = int(start_date_short[11:13])
sd_sec = int(start_date_short[13:15])

fd_year = int(finish_date_short[0:4])
fd_month = int(finish_date_short[4:6])
fd_day = int(finish_date_short[6:8])
fd_hour = int(finish_date_short[9:11])
fd_min = int(finish_date_short[11:13])
fd_sec = int(finish_date_short[13:15])

In [ ]:
start = datetime.datetime(year=sd_year, month=sd_month, day=sd_day, hour=sd_hour, minute=sd_min,second=sd_sec)
end = datetime.datetime(year=fd_year, month=fd_month, day=fd_day, hour=fd_hour, minute=fd_min,second=fd_sec)
diff = end - start
print(diff)

In [ ]:
import smtplib
from email.message import EmailMessage

msg_subject = "[GG Colab] Model {}".format(start_date_short)
msg_content = "It take {} for training.".format(diff)

msg = EmailMessage()
msg['Subject'] = msg_subject
msg['From'] = '*'
msg['To']='ptmhoang97@gmail.com'
msg.set_content(msg_content)

server = smtplib.SMTP_SSL('smtp.gmail.com',465)
server.login("ptmhoang97@gmail.com","euclxykrqkkrquqz")
server.send_message(msg)
server.quit()